#**Trending Topics Project**
* This project is aimed at developing dashboard visualisation of the Twitter trending topics in different geographical locations.
* The visualizations provide also delves into sentiment analysis of the trending topics to understand whether they are positive or negative.
* The categorization of the topics is necessary to ensure we enhance our visibility in the appropriate trends.
* This project will involve developing a model to read and automate prediction of sentiments from trending topics.

**Tools and Libraries**
* Google Colab: hosting the code, training, testing and evaluating the model.
* Tweepy: packages to access Twitter API and pull tweets.
* Twitter’s API - We shall use Kiboi's Developer Account
* Emojis: package for conversion of emojis into text and vice versa
* Trax / Keras: building a deep neural network.
* Google Cloud Compute Engine: a virtual machine in the cloud to host the script.
* Google Cloud Storage: a bucket in the cloud to store the data.
* Google Cloud Secrets Manager: a place in the cloud to store our secrets and keys.
* Google Data Studio: an interactive dashboard studio (We shall use 247rpwr@gmail.com account)



In [1]:
!pip install emojis
!pip install trax
!pip install tweepy
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 15.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 5.0 MB/s eta 0:00:00


In [2]:
# add to system path
import sys
sys.path.append('/kaggle/working/mysitepackages')

Let's import the necessary libraries and packages

In [3]:
import tweepy                             #package to conveniently access the Twitter API with Python
import json                               #a standard text-based format for representing structured data based on JavaScript object syntax.

import warnings
warnings.filterwarnings("ignore")

#import geocoder
import pandas as pd
import numpy as np

import os 
import time
import emojis


import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import random as rnd                       # pseudo-random number generator

import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings


# import relevant libraries
import trax

# import trax.fastmath.numpy

import trax.fastmath.numpy as np


# import trax.layers
from trax import layers as tl


from trax.supervised import training


In [12]:
    api_key= 'SOmTAIoDSCqOeR3cFaJEt5kOe'
    api_secret= 'dqZL0R6kdaCpIMkDBxYCpBbZ5VHLrO3sNBb83vYef3Tq0lNIvC'
    access_token= '912397733377581056-eZz0fQYAA8uVa3gdapd9s2Xg0ASgwtj'
    access_token_secret= 'nlQ8KJxMswrXcCd5p5wC4B2w2yvxN67aaJPcoCb1C9yhY'

In [13]:
# authorize api handshake
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

## Downlaod training data

In [14]:
# downloads sample twitter dataset. uncomment the line below if running on a local machine.
nltk.download('twitter_samples') 

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [15]:
# select the set of positive and negative tweets
nltk_positive_tweets = twitter_samples.strings('positive_tweets.json')
nltk_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [16]:
import pandas as pd

file_path = '/kaggle/input/data-tweets/Training Tweets.csv'
tweets = pd.read_csv(file_path, encoding = "ISO-8859-1", names = ["target", "ids", "date", "flag", "user", "text"])
tweets.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [18]:
tweets['target'].unique()

array([0, 4])

In [19]:
fmap = {4:1, 0:0}

tweets['target'] = tweets['target'] .map(fmap)

tweets['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [20]:
pos_tweets = list(tweets[tweets['target'] == 1]['text'])
all_positive_tweets = nltk_positive_tweets + pos_tweets

neg_tweets = list(tweets[tweets['target'] == 0]['text'])
all_negative_tweets = nltk_negative_tweets + neg_tweets

In [21]:
print('Number of positive tweets: ', len(all_positive_tweets))
print('Number of negative tweets: ', len(all_negative_tweets))

Number of positive tweets:  805000
Number of negative tweets:  805000


In [22]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[rnd.randint(0,805000)])

@fionajc3 lol....oh okiday  That works too 


In [23]:
# print negative in red
print('\033[91m' + all_negative_tweets[rnd.randint(0,805000)])

IDK how many of us are being "doctored" with this toxin. when will we have the freedom to go organic :( https://t.co/LGCx3ifqC7


In [24]:
percentage = (len(all_positive_tweets[:724500]) / len(all_positive_tweets)) * 100
print("percentage used in training:", percentage)

percentage = (len(all_positive_tweets[724500:]) / len(all_positive_tweets)) * 100
print("percentage used for validation:", percentage)

percentage used in training: 90.0
percentage used for validation: 10.0


In [25]:
# Split positive set into validation and training
val_pos   = all_positive_tweets[724500:] # generating validation set for positive tweets
train_pos  = all_positive_tweets[:724500]# generating training set for positive tweets

# Split negative set into validation and training
val_neg   = all_negative_tweets[724500:] # generating validation set for negative tweets
train_neg  = all_negative_tweets[:724500] # generating training set for nagative tweets

# Combine training data into one set
train_x = train_pos + train_neg 

# Combine validation data into one set
val_x  = val_pos + val_neg

# Set the labels for the training set (1 for positive, 0 for negative)
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))

# Set the labels for the validation set (1 for positive, 0 for negative)
val_y  = np.append(np.ones(len(val_pos)), np.zeros(len(val_neg)))

print(f"Length of train_x:  {len(train_x)}")
print(f"Length of val_x:  {len(val_x)}")

Length of train_x:  1449000
Length of val_x:  161000


**Pre-process Tweets**

In [26]:
# Our selected sample. Complex enough to exemplify each step
tweet = all_positive_tweets[100]
print(tweet)

@metalgear_jp @Kojima_Hideo I want you're T-shirts ! They are so cool ! :D


In [27]:
# download the stopwords from NLTK
nltk.download('stopwords')

#Import the english stop words list from NLTK
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
import re
import string
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import emojis

def process_tweet(tweet):
    # Remove old style retweet text "RT"
    new_tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # Decode emojis to text descriptions
    new_tweet = emojis.decode(new_tweet)

    # Remove hyperlinks
    new_tweet = re.sub(r'http\S+|www\.[^\s]+', '', new_tweet)
    
    # Remove hashtags and underscores
    new_tweet = re.sub(r'#[^\s]+|_', '', new_tweet)

    # Remove all numbers
    new_tweet = re.sub(r'\d+', '', new_tweet)

    # Remove usernames
    new_tweet = re.sub(r'@[^\s]+', '', new_tweet)
    
    # Remove punctuation even in the middle of a string "in.the.middle"
    new_tweet = re.sub(r'[^\w\s]', ' ', new_tweet)
    
    # Convert text to lowercase
    new_tweet = new_tweet.lower()

    # Instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

    # Tokenize tweets
    tweet_tokens = tokenizer.tokenize(new_tweet)

    stopwords_english = set(stopwords.words('english'))
    
    # Remove stopwords and punctuation
    tweets_clean = [word for word in tweet_tokens if word not in stopwords_english and word not in string.punctuation]

    # Instantiate stemming class
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    tweets_stem = [stemmer.stem(word) for word in tweets_clean]  # Stemming word and append to the list
    
    return tweets_stem


In [29]:
tweet = all_positive_tweets[500]
print(f'Original tweet:     {tweet}')

processed_tweet = process_tweet(tweet)
print(f'Processed tweet:     {processed_tweet}')

Original tweet:     Dang that is some rad @AbzuGame #fanart! :D https://t.co/bI8k8tb9ht
Processed tweet:     ['dang', 'rad']


**Build the Vocabulary**

In [30]:
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

In [31]:
# # Build the vocabulary

# # Include special tokens 
# # started with pad, end of line and unk tokens

# # Note that we build vocab using training data
# for tweet in train_x: 
#     processed_tweet = process_tweet(tweet)
#     for word in processed_tweet:
#         if word not in Vocab: 
#             Vocab[word] = len(Vocab)
    
# print("Total words in vocab are", len(Vocab), '\n')

# first_10 = {k: Vocab[k] for k in sorted(Vocab.keys())[:10]}
# display(first_10)

In [32]:
json.dump(Vocab, open( "Vocab.json", 'w' ) )

**Build a Tensor from a Tweet**

In [33]:
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''  

    # Process the tweet into a list of words
    # where only important words are kept (stop words removed)
    word_l = process_tweet(tweet)
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
        
    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = []
    
    # Get the unique integer ID of the __UNK__ token
    unk_ID = vocab_dict[unk_token]
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    # for each word in the list:
    for word in word_l:
        
        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.
        word_ID = vocab_dict.get(word, unk_ID)

        
        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID) 
    
    return tensor_l

In [34]:
print("Actual tweet is\n", val_pos[1])
print("\nTensor of tweet:\n", tweet_to_tensor(val_pos[1], vocab_dict=Vocab))

Actual tweet is
 4 hours and counting 

Tensor of tweet:
 [2, 2]


**Build a Batch Generator**

In [35]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of posstive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example
        
    '''     

    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from pos_index and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset lenght,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            # get the tweet as pos_index
            tweet = data_pos[pos_index_lines[pos_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1


        # Second part: Pack n_to_take negative examples
    
        # Using the same batch list, start from neg_index and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset length,
            if neg_index >= len_data_neg:
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
            # get the tweet as neg_index
            tweet = data_neg[neg_index_lines[neg_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index = neg_index + 1

        if stop:
            break;

        # Update the start index for positive data 
        # so that it's n_to_take positions after the current pos_index
        pos_index += n_to_take
        
        # Update the start index for negative data 
        # so that it's n_to_take positions after the current neg_index
        neg_index += n_to_take
        
        # Get the max tweet length (the length of the longest tweet) 
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:

            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = [0] * n_pad
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor +  pad_l

            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = [1] * n_to_take
        
        # Generate the list of targets for the negative examples (a list of zeros)
        # The length is the number of negative examples in the batch
        target_neg = [0] * n_to_take
        
        # Concatenate the positve and negative targets
        target_l = target_pos + target_neg
        
        # Convert the target list into a numpy array
        targets = np.array(target_l)

        # Example weights: Treat all examples equally importantly.It should return an np.array. Hint: Use np.ones_like()
        example_weights = np.ones_like(targets)
        
        # note we use yield and not return
        yield inputs, targets, example_weights

In [36]:
# Set the random number generator for the shuffle procedure
rnd.seed(42) 

# Create the training data generator
def train_generator(batch_size, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def test_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, False, Vocab, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


**Build a Model**

In [37]:
def classifier(vocab_size=len(Vocab), embedding_dim=256, output_dim=2, mode='train'):
        

    # create embedding layer
    embed_layer = tl.Embedding(
        vocab_size=vocab_size, # Size of the vocabulary
        d_feature=embedding_dim)  # Embedding dimension
    
    # Create a mean layer, to create an "average" word embedding
    mean_layer = tl.Mean(axis=1)
    
    # Create a dense layer, one unit for each output
    dense_output_layer = tl.Dense(n_units = output_dim)

    # Create the log softmax layer (no parameters needed)
    log_softmax_layer = tl.LogSoftmax()
    
    # Use tl.Serial to combine all layers
    # and create the classifier
    # of type trax.layers.combinators.Serial
    model = tl.Serial(
      embed_layer, # embedding layer
      mean_layer, # mean layer
      dense_output_layer, # dense output layer 
      log_softmax_layer # log softmax layer
    )

    # return the model of type
    return model

In [38]:
tmp_model = classifier()

print(type(tmp_model))
display(tmp_model)

<class 'trax.layers.combinators.Serial'>


Serial[
  Embedding_3_256
  Mean
  Dense_2
  LogSoftmax
]

**Train the Model**

In [39]:
batch_size = 16
rnd.seed(42)

train_task = training.TrainTask(
    labeled_data=train_generator(batch_size=batch_size, shuffle=True),
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.0001),
    n_steps_per_checkpoint=100,
)

eval_task = training.EvalTask(
    labeled_data=val_generator(batch_size=batch_size, shuffle=True),
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)

model = classifier()

In [40]:
! pwd

/kaggle/working


In [41]:
output_dir = '~/kaggle/working'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

/root/kaggle/working


In [42]:
def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    '''
    Input: 
        classifier - the model you are building
        train_task - Training task
        eval_task - Evaluation task
        n_steps - the evaluation steps
        output_dir - folder to save your files
    Output:
        trainer -  trax trainer
    '''

    training_loop = training.Loop(
                                classifier, # The learning model
                                train_task, # The training task
                                eval_tasks = eval_task, # The evaluation task
                                output_dir = output_dir) # The output directory

    training_loop.run(n_steps = n_steps)

    # Return the training_loop, since it has the model.
    return training_loop

In [43]:
training_loop = train_model(model, train_task, eval_task, 100, output_dir_expand)


Step      1: Total number of trainable weights: 1282
Step      1: Ran 1 train steps in 0.48 secs
Step      1: train CrossEntropyLoss |  0.69489336
Step      1: eval  CrossEntropyLoss |  0.69343710
Step      1: eval          Accuracy |  0.50000000

Step    100: Ran 99 train steps in 5.82 secs
Step    100: train CrossEntropyLoss |  0.69331938
Step    100: eval  CrossEntropyLoss |  0.69890332
Step    100: eval          Accuracy |  0.37500000


**Testing the Model**

In [44]:
# Create a generator object
tmp_train_generator = train_generator(16)

# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 13) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.


In [45]:
# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


Array([[-0.69836026, -0.68796116],
       [-0.675672  , -0.71093315],
       [-0.6907396 , -0.6955606 ],
       [-0.7137753 , -0.67293596],
       [-0.66822505, -0.71870637],
       [-0.70989984, -0.6766706 ],
       [-0.675672  , -0.71093315],
       [-0.6719413 , -0.7148126 ],
       [-0.70989984, -0.6766706 ],
       [-0.686951  , -0.699382  ],
       [-0.7021923 , -0.6841831 ],
       [-0.7021923 , -0.6841831 ],
       [-0.69836026, -0.68796116],
       [-0.7021923 , -0.6841831 ],
       [-0.70989984, -0.6766706 ],
       [-0.686951  , -0.699382  ]], dtype=float32)

In [46]:
# turn probabilites into category predictions
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {tmp_pred[i,0]:.4f}\tPos log prob {tmp_pred[i,1]:.4f}\t is positive? {p}\t actual {tmp_targets[i]}")

Neg log prob -0.6984	Pos log prob -0.6880	 is positive? True	 actual 1
Neg log prob -0.6757	Pos log prob -0.7109	 is positive? False	 actual 1
Neg log prob -0.6907	Pos log prob -0.6956	 is positive? False	 actual 1
Neg log prob -0.7138	Pos log prob -0.6729	 is positive? True	 actual 1
Neg log prob -0.6682	Pos log prob -0.7187	 is positive? False	 actual 1
Neg log prob -0.7099	Pos log prob -0.6767	 is positive? True	 actual 1
Neg log prob -0.6757	Pos log prob -0.7109	 is positive? False	 actual 1
Neg log prob -0.6719	Pos log prob -0.7148	 is positive? False	 actual 1
Neg log prob -0.7099	Pos log prob -0.6767	 is positive? True	 actual 0
Neg log prob -0.6870	Pos log prob -0.6994	 is positive? False	 actual 0
Neg log prob -0.7022	Pos log prob -0.6842	 is positive? True	 actual 0
Neg log prob -0.7022	Pos log prob -0.6842	 is positive? True	 actual 0
Neg log prob -0.6984	Pos log prob -0.6880	 is positive? True	 actual 0
Neg log prob -0.7022	Pos log prob -0.6842	 is positive? True	 actual 0


**Build Accuracy Function & Test on One Batch of Data**

In [47]:
def compute_accuracy(preds, y, y_weights):
    """
    Input: 
        preds: a tensor of shape (dim_batch, output_dim) 
        y: a tensor of shape (dim_batch, output_dim) with the true labels
        y_weights: a n.ndarray with the a weight for each example
    Output: 
        accuracy: a float between 0-1 
        weighted_num_correct (np.float32): Sum of the weighted correct predictions
        sum_weights (np.float32): Sum of the weights
    """

    # Create an array of booleans, 
    # True if the probability of positive sentiment is greater than
    # the probability of negative sentiment
    # else False
    is_pos =  preds[:,1] > preds[:,0]

    # convert the array of booleans into an array of np.int32
    is_pos_int = np.array(is_pos, dtype = np.int32)
    
    # compare the array of predictions (as int32) with the target (labels) of type int32
    correct = is_pos_int == y

    # Count the sum of the weights.
    sum_weights = np.sum(y_weights)
    
    # convert the array of correct predictions (boolean) into an arrayof np.float32
    correct_float = np.array(correct, dtype = np.float32)
    
    # Multiply each prediction with its corresponding weight.
    weighted_correct_float = correct_float * y_weights

    # Sum up the weighted correct predictions (of type np.float32), to go in the
    # denominator.
    weighted_num_correct = np.sum(weighted_correct_float)
 
    # Divide the number of weighted correct predictions by the sum of the
    # weights.
    accuracy = sum_weights / weighted_num_correct 

    return accuracy, weighted_num_correct, sum_weights

In [48]:
# test your function
tmp_val_generator = val_generator(64)

# get one batch
tmp_batch = next(tmp_val_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)

tmp_acc, tmp_num_correct, tmp_num_predictions = compute_accuracy(preds=tmp_pred, y=tmp_targets, y_weights=tmp_example_weights)

print(f"Model's prediction accuracy on a single training batch is: {100 * tmp_acc}%")
print(f"Weighted number of correct predictions {tmp_num_correct}; weighted number of total observations predicted {tmp_num_predictions}")

Model's prediction accuracy on a single training batch is: 228.57144165039062%
Weighted number of correct predictions 28.0; weighted number of total observations predicted 64


**Model Evaluation**

In [49]:
def test_model(generator, model):
    '''
    Input: 
        generator: an iterator instance that provides batches of inputs and targets
        model: a model instance 
    Output: 
        accuracy: float corresponding to the accuracy
    '''
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0

    for batch in generator: 
        
        # Retrieve the inputs from the batch
        inputs = batch[0]
        
        # Retrieve the targets (actual labels) from the batch
        targets = batch[1]
        
        # Retrieve the example weight.
        example_weight = batch[2]

        # Make predictions using the inputs
        pred = model(inputs)
        
        # Calculate accuracy for the batch by comparing its predictions and targets
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(preds=pred, y=targets, y_weights=example_weight)
        
        # Update the total number of correct predictions
        # by adding the number of correct predictions from this batch
        total_num_correct += batch_num_correct
        
        # Update the total number of predictions 
        # by adding the number of predictions made for the batch
        total_num_pred += batch_num_pred

    # Calculate accuracy over all examples
    accuracy = total_num_correct / total_num_pred

    return accuracy

In [50]:
# testing the accuracy of your model: this takes around 20 seconds
model = training_loop.eval_model

accuracy = test_model(test_generator(16), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.5143


#**Getting Live Twitter Data**

In [51]:
kenya_woeid = 23424863

# Retrieve top 50 trending topics for Kenya
trending_topics = api.get_place_trends(id=kenya_woeid, exclude="hashtags")

topic_list = []

# Loop through each trending topic and create a list
for trend in trending_topics[0]['trends']:
    topic = trend['name']
    topic_list.append(topic)
    print(topic)
auth_params = {
    'app_key': 'SOmTAIoDSCqOeR3cFaJEt5kOe',
    'app_secret': 'dqZL0R6kdaCpIMkDBxYCpBbZ5VHLrO3sNBb83vYef3Tq0lNIvC',
    'oauth_token': '912397733377581056-eZz0fQYAA8uVa3gdapd9s2Xg0ASgwtj',
    'oauth_token_secret': 'nlQ8KJxMswrXcCd5p5wC4B2w2yvxN67aaJPcoCb1C9yhY',
}

NHIF
blessed sunday
Pole
Leicester
duale book signing
transact na im bank
Uganda
Happy New Week
okiya omtatah
Leeds
Everton
Granit Xhaka
Poch
Erdogan
Counties HealthCare
unlocking africa gateway
Mainz
save kenyan donkeys
Mary Moraa
Orengo
Condolences
Yanga
Turkey
Southampton
Airtel
Donald
Sonko
pastor fred
AfCFTA
Russia
Siaya
Anti-Homosexuality Act 2023
Brentford
Busia
Weghorst
Shiv
Bola Tinubu
Museveni
Maddison
Luton
Ocon
Lewis Hall
Fabrizio Romano
Oduol
Dortmund


In [56]:
!pip install --upgrade tweepy

In [58]:
program_start = time.time()            # time the program

max_tweets = 20        # number of tweets to grab for each topic
n = 20                 # top n number of retweets to keep

#assert (max_tweets * len(df) * 50) <= 100000 , 'Request Limit Assert Error: Too many tweets! \n \n Twitter has a 24 hour limit of 100,000 requests \n \n Try changing "max_tweets"'    
    
columns = ['username', 'user_location', 'retweetcount', 'favorites', 'text',   # set up columns for dataframes
           'hashtags', 'emojis', 'trending_topic', 'search_type']          

recent_tweets_data = pd.DataFrame(columns = columns)                           # create empty dataframe    
    

for k in range(len(trending_topics[0]['trends'])):                # loop though each trending topic for each town
    
    run_start = time.time()                                       # start time of the run for each topic   

    topic = trending_topics[0]['trends'][k]['name']               # the trending topic or hashtag (1 out of 50)  
                                                        
                                                                            # grab tweets with Cursor
    tweets = tweepy.Cursor(api.search_tweets, q = topic,                           # search for each trending topic                                 
                     lang="en", result_type = 'recent' ,                    # most recent tweets in english
                      tweet_mode = 'extended').items(max_tweets)            # longer tweets and grab max_tweets number of tweets
    
    tweet_list = [tweet for tweet in tweets]                                # create list of tweets
                
    recent_tweets_topic = pd.DataFrame(columns = columns)         # create dataframe to put in current top tweets for this town and trending topic
        
    for tweet in tweet_list:                                      # loop through each tweet that was grabbed
        
        username = tweet.user.screen_name              # store username
        user_location = tweet.user.location            # store location of user
        retweetcount = tweet.retweet_count             # store retweet count
        favorites = tweet.favorite_count               # store favorite count
        hashtags = tweet.entities['hashtags']          # store hashtags
        search_type = 'recent'
    
        try:                              
            text = tweet.retweeted_status.full_text    # store text if it's a retweet
        
        except AttributeError: 
            text = tweet.full_text                     # store text if it's a regular tweet
            
        emoji = list(emojis.get(text))                      # get the emojis
        
        curr_tweet = [username, user_location, retweetcount, favorites, text,            # store current tweet's data in a list soon to be a row
                                hashtags, emoji, topic, search_type]                             
        
        recent_tweets_topic.loc[len(recent_tweets_topic)] = curr_tweet                   # add current tweet data to dataframe for town and topic
                            
    recent_tweets_topic.sort_values(by=['retweetcount', 'favorites'], inplace = True, ascending = False)        # sort the retweet values highest first
                            
    top_n = recent_tweets_topic[0:n]                                                     # keep only the top n
                            
    recent_tweets_data = pd.concat([recent_tweets_data, top_n], ignore_index = True, sort = False)              # concatenate top n to final dataframe
    
    run_end = time.time()                                                              # stop the time for the run
    run_duration = round((run_end - run_start)/60, 2)                                  # calculate duration
    print(f'Time for {topic} : {run_duration} mins')                                   # print duration       
    

program_end = time.time()                                                               # stop the program time
program_duration = round(((program_end - program_start) / 60), 2)                       # calculate duration
print(f'\n \n ************************************************** \n \n TOTAL TIME TO GRAB TWEETS : \t {program_duration} mins')    # print duration as total time
print('\n **************************************************')    


Time for NHIF : 0.01 mins
Time for blessed sunday : 0.01 mins
Time for Pole : 0.01 mins
Time for Leicester : 0.01 mins
Time for duale book signing : 0.01 mins
Time for transact na im bank : 0.01 mins
Time for Uganda : 0.01 mins
Time for Happy New Week : 0.01 mins
Time for okiya omtatah : 0.01 mins
Time for Leeds : 0.02 mins
Time for Everton : 0.01 mins
Time for Granit Xhaka : 0.01 mins
Time for Poch : 0.01 mins
Time for Erdogan : 0.01 mins
Time for Counties HealthCare : 0.01 mins
Time for unlocking africa gateway : 0.01 mins
Time for Mainz : 0.01 mins
Time for save kenyan donkeys : 0.01 mins
Time for Mary Moraa : 0.01 mins
Time for Orengo : 0.01 mins
Time for Condolences : 0.01 mins
Time for Yanga : 0.01 mins
Time for Turkey : 0.01 mins
Time for Southampton : 0.01 mins
Time for Airtel : 0.01 mins
Time for Donald : 0.01 mins
Time for Sonko : 0.01 mins
Time for pastor fred : 0.01 mins
Time for AfCFTA : 0.01 mins
Time for Russia : 0.01 mins
Time for Siaya : 0.01 mins
Time for Anti-Homosex

In [59]:
recent_tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,NdichuMburu,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent
1,shadrac_matata,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent
2,DonAntonioh,,320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent
3,NonieMG,Kenya,258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent
4,anmugambi,"San Francisco, CA",258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent


In [60]:
len(recent_tweets_data)

900

In [61]:
program_start = time.time()            # time the program

max_tweets = 20        # number of tweets to grab for each topic
n = 20                 # top n number of retweets to keep

#assert (max_tweets * len(df) * 50) <= 100000 , 'Request Limit Assert Error: Too many tweets! \n \n Twitter has a 24 hour limit of 100,000 requests \n \n Try changing "max_tweets"'


columns = ['username', 'user_location', 'retweetcount', 'favorites', 'text',   # set up columns for dataframes
           'hashtags', 'emojis', 'trending_topic', 'search_type']          

pop_tweets_data = pd.DataFrame(columns = columns)                 # create empty dataframe
    

for q in range(len(trending_topics[0]['trends'])):                # loop though each trending topic for each town

    topic = trending_topics[0]['trends'][q]['name']               # the trending topic or hashtag (1 out of 50)  
                                                                           
                                                                            # grab tweets with Cursor
    tweets = tweepy.Cursor(api.search_tweets, q = topic,                           # search for each trending topic                                 
                     lang="en", result_type = 'popular' ,                   # most recent tweets in english
                      tweet_mode = 'extended').items(max_tweets)            # longer tweets and grab max_tweets number of tweets
    
    tweet_list = [tweet for tweet in tweets]                                # create list of tweets
                
    pop_tweets_topic = pd.DataFrame(columns = columns)         # create dataframe to put in current top tweets for this town and trending topic
        
    for tweet in tweet_list:                                   # loop through each tweet that was grabbed
        
        username = tweet.user.screen_name              # store username
        user_location = tweet.user.location            # store location of user
        retweetcount = tweet.retweet_count             # store retweet count
        favorites = tweet.favorite_count               # store favorite count
        hashtags = tweet.entities['hashtags']          # store hashtags
        search_type = 'popular'
        
        try:                              
            text = tweet.retweeted_status.full_text    # store text if it's a retweet
        
        except AttributeError: 
            text = tweet.full_text                     # store text if it's a regular tweet
            
        emoji = list(emojis.get(text))                      # get the emojis    
        
        curr_tweet = [username, user_location, retweetcount, favorites, text,      # store current tweet's data in a list soon to be a row
                                hashtags, emoji, topic, search_type]                             
        
        pop_tweets_topic.loc[len(pop_tweets_topic)] = curr_tweet                   # add current tweet data to dataframe for town and topic
                            
    pop_tweets_topic.sort_values(by=['retweetcount', 'favorites'], inplace = True, ascending = False)        # sort the retweet values highest first
                            
    top_n = pop_tweets_topic[0:n]                                                                  # keep only the top n
                            
    pop_tweets_data = pd.concat([pop_tweets_data, top_n], ignore_index = True, sort = False)       # concatenate top n to final dataframe
            
    
program_end = time.time()                                                               # stop the program time
program_duration = round(((program_end - program_start) / 60), 2)                       # calculate duration
print(f'\n \n ************************************************** \n \n TOTAL TIME TO GRAB TWEETS : \t {program_duration} mins')    # print duration as total time
print('\n **************************************************')    


 
 ************************************************** 
 
 TOTAL TIME TO GRAB TWEETS : 	 12.17 mins

 **************************************************


In [62]:
pop_tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,NationAfrica,Kenya,9,35,"According to doctors, it is a miracle that Mr ...","[{'text': 'HealthyNation', 'indices': [207, 22...",[],NHIF,popular
1,Beard_Vet,"Omaha, NE",491,2074,"Good morning to Everyone, except the FBI who F...",[],"[🇺🇸, ☕, 👊]",blessed sunday,popular
2,cbs_ke,"Nairobi, Kenya",113,609,"Heavenly Father, \nThank you for this blessed ...",[],[],blessed sunday,popular
3,PrayTheRosary12,,109,1125,Blessed Sunday in the Octave of the Ascension/...,[],"[🙏, ✝️]",blessed sunday,popular
4,ZaakeFrancis,"Kampala, Uganda",57,661,Naye kale kyovolaba omuntu affa stress.. \nWam...,"[{'text': 'jogo', 'indices': [121, 126]}]",[💪],blessed sunday,popular


In [63]:
len(pop_tweets_data)

699

In [64]:
pop_tweets_data['trending_topic'].value_counts()

Turkey                         20
Condolences                    20
Lewis Hall                     20
Ocon                           20
Luton                          20
Maddison                       20
Museveni                       20
Bola Tinubu                    20
Shiv                           20
Weghorst                       20
Brentford                      20
Siaya                          20
Russia                         20
Donald                         20
Airtel                         20
Southampton                    20
Dortmund                       20
Erdogan                        20
Uganda                         20
Everton                        20
Mainz                          20
Pole                           20
Granit Xhaka                   20
Leicester                      20
Happy New Week                 20
Leeds                          20
Busia                          17
Oduol                          16
Poch                           16
blessed sunday

In [65]:
tweets_data = pd.concat([recent_tweets_data, pop_tweets_data], ignore_index=True)

tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,NdichuMburu,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent
1,shadrac_matata,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent
2,DonAntonioh,,320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent
3,NonieMG,Kenya,258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent
4,anmugambi,"San Francisco, CA",258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent


In [66]:
tweets_data.tail()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
1594,brfootball,,635,6565,Dortmund fans welcome their team in numbers 💥\...,[],[💥],Dortmund,popular
1595,goal,Everywhere,611,14018,KOLN EQUALISE AGAINST BAYERN 😱\n\nIT'S BACK IN...,[],"[🍿, 😱]",Dortmund,popular
1596,ErlingRoIe,Back 2 Back Champions,539,7085,That Dortmund game reminded me. We see things ...,[],[],Dortmund,popular
1597,433,Get the 433 app! ➡️,535,21362,Last game for Dortmund? 💔 https://t.co/kpLKXnxoZ9,[],[💔],Dortmund,popular
1598,PoojaMedia,Nigeria,369,3717,Bundesliga took the original trophy to Dortmun...,[],[😭],Dortmund,popular


In [67]:
len(tweets_data)

1599

**Use Models on Live Twitter Data**

In [68]:
# this is used to predict on your own sentnece
def predict(sentence):
    inputs = np.array(tweet_to_tensor(sentence, vocab_dict=Vocab))
    
    # Batch size 1, add dimension for batch, to work with the model
    inputs = inputs[None, :]  
    
    # predict with the model
    preds_probs = model(inputs)
    
    # Turn probabilities into categories
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sentiment = "negative"
    if preds == 1:
        sentiment = 'positive'

    return sentiment, round(float(preds_probs[0, 0]),4), round(float(preds_probs[0, 1]),4)

In [69]:
for i in range(len(tweets_data)):

    tweets_data.loc[i, 'sentiment'], tweets_data.loc[i, 'neg_prob'], tweets_data.loc[i, 'pos_prob'] = predict(tweets_data['text'].iloc[i])

In [70]:
tweets_data.head(20)

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type,sentiment,neg_prob,pos_prob
0,NdichuMburu,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent,negative,-0.6682,-0.7187
1,shadrac_matata,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent,negative,-0.6682,-0.7187
2,DonAntonioh,,320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent,negative,-0.6682,-0.7187
3,NonieMG,Kenya,258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent,negative,-0.6682,-0.7187
4,anmugambi,"San Francisco, CA",258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent,negative,-0.6682,-0.7187
5,Dendricck,Dark void,258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent,negative,-0.6682,-0.7187
6,AntoGachau,"Nairobi, Kenya",258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent,negative,-0.6682,-0.7187
7,lindahoguttu,Kenya,258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent,negative,-0.6682,-0.7187
8,Mwirigi,"Nairobi, Kenya",258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent,negative,-0.6682,-0.7187
9,mashjayk,You crane,258,0,If @nhifkenya has ever paid your hospital bill...,[],[],NHIF,recent,negative,-0.6682,-0.7187


In [71]:
max_pos = max(tweets_data['pos_prob'])

pos_tweets  = tweets_data.loc[tweets_data['pos_prob'] == max_pos][['text']]

for p in pos_tweets['text']:
    print(p) 

Bill ya 1m NHIF inalipa 8K. I've paid for it religiously for the last 8 years. 
Naaahhh man. Iko shida
Bill ya 1m NHIF inalipa 8K. I've paid for it religiously for the last 8 years. 
Naaahhh man. Iko shida
Bill ya 1m NHIF inalipa 8K. I've paid for it religiously for the last 8 years. 
Naaahhh man. Iko shida
If @nhifkenya has ever paid your hospital bill. Kindly comment down here. Coz  when I needed it it only paid 1% of the total bill. 1 %. Never missed a payment in NHIF.
What is this. 
I am angry. I want answers. 
Kazi yenu Ni kulipa ad kwa TV
If @nhifkenya has ever paid your hospital bill. Kindly comment down here. Coz  when I needed it it only paid 1% of the total bill. 1 %. Never missed a payment in NHIF.
What is this. 
I am angry. I want answers. 
Kazi yenu Ni kulipa ad kwa TV
If @nhifkenya has ever paid your hospital bill. Kindly comment down here. Coz  when I needed it it only paid 1% of the total bill. 1 %. Never missed a payment in NHIF.
What is this. 
I am angry. I want answe

In [72]:
max_neg = max(tweets_data['neg_prob'])

neg_tweets  = tweets_data.loc[tweets_data['neg_prob'] == max_neg][['text']]

for n in neg_tweets['text']:
    print(n) 


Bill ya 1m NHIF inalipa 8K. I've paid for it religiously for the last 8 years. 
Naaahhh man. Iko shida
Bill ya 1m NHIF inalipa 8K. I've paid for it religiously for the last 8 years. 
Naaahhh man. Iko shida
Bill ya 1m NHIF inalipa 8K. I've paid for it religiously for the last 8 years. 
Naaahhh man. Iko shida
If @nhifkenya has ever paid your hospital bill. Kindly comment down here. Coz  when I needed it it only paid 1% of the total bill. 1 %. Never missed a payment in NHIF.
What is this. 
I am angry. I want answers. 
Kazi yenu Ni kulipa ad kwa TV
If @nhifkenya has ever paid your hospital bill. Kindly comment down here. Coz  when I needed it it only paid 1% of the total bill. 1 %. Never missed a payment in NHIF.
What is this. 
I am angry. I want answers. 
Kazi yenu Ni kulipa ad kwa TV
If @nhifkenya has ever paid your hospital bill. Kindly comment down here. Coz  when I needed it it only paid 1% of the total bill. 1 %. Never missed a payment in NHIF.
What is this. 
I am angry. I want answe

In [73]:
difficult = "Great!! I got a flat tire!!"

predict(difficult)

('negative', -0.6682, -0.7187)

In [74]:
pred_positive_tweets = tweets_data.sort_values(by=['pos_prob'], inplace = False, ascending = False)       
                            
top_positive = pred_positive_tweets[0:25]     

top_positive

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type,sentiment,neg_prob,pos_prob
0,NdichuMburu,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent,negative,-0.6682,-0.7187
1074,Gunnersc0m,"Highbury, London",134,1242,This will be Granit Xhaka today ❤\n\nhttps://t...,[],[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1072,3cbPerformance,"Los Angeles, CA",137,994,If today is indeed Granit Xhaka’s last match w...,"[{'text': 'Arsenal', 'indices': [197, 205]}]",[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1071,Gunnersc0m,"Highbury, London",254,4336,I’m gonna need Granit Xhaka to do this today\n...,[],[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1070,BBCSport,"MediaCityUK, Salford",355,4880,Granit Xhaka scores for Arsenal on what could ...,"[{'text': 'BBCFootball', 'indices': [109, 121]...","[🔥, 🔴]",Granit Xhaka,popular,negative,-0.6682,-0.7187
1069,afcstuff,,451,6930,Granit Xhaka’s matchday post on Instagram stor...,"[{'text': 'afc', 'indices': [130, 134]}]",[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1068,afcstuff,,653,11227,Granit Xhaka getting emotional after his subst...,"[{'text': 'afc', 'indices': [113, 117]}]",[❤️],Granit Xhaka,popular,negative,-0.6682,-0.7187
1067,goal,Everywhere,710,14768,Granit Xhaka scores the first goal in Arsenal'...,[],[❤️],Granit Xhaka,popular,negative,-0.6682,-0.7187
1066,piersmorgan,,748,10380,"Thank you and goodbye, Granit. We had a few wo...","[{'text': 'xhaka', 'indices': [225, 231]}, {'t...",[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1065,ESPNFC,,765,12854,Granit Xhaka records a brace in 15 minutes in ...,[],"[🥺, 👏]",Granit Xhaka,popular,negative,-0.6682,-0.7187


In [75]:
pred_negative_tweets = tweets_data.sort_values(by=['neg_prob'], inplace = False, ascending = False)        
                            
top_negative = pred_negative_tweets[0:25]     

top_negative

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type,sentiment,neg_prob,pos_prob
0,NdichuMburu,"Nairobi, Kenya",320,0,Bill ya 1m NHIF inalipa 8K. I've paid for it r...,[],[],NHIF,recent,negative,-0.6682,-0.7187
1074,Gunnersc0m,"Highbury, London",134,1242,This will be Granit Xhaka today ❤\n\nhttps://t...,[],[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1072,3cbPerformance,"Los Angeles, CA",137,994,If today is indeed Granit Xhaka’s last match w...,"[{'text': 'Arsenal', 'indices': [197, 205]}]",[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1071,Gunnersc0m,"Highbury, London",254,4336,I’m gonna need Granit Xhaka to do this today\n...,[],[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1070,BBCSport,"MediaCityUK, Salford",355,4880,Granit Xhaka scores for Arsenal on what could ...,"[{'text': 'BBCFootball', 'indices': [109, 121]...","[🔥, 🔴]",Granit Xhaka,popular,negative,-0.6682,-0.7187
1069,afcstuff,,451,6930,Granit Xhaka’s matchday post on Instagram stor...,"[{'text': 'afc', 'indices': [130, 134]}]",[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1068,afcstuff,,653,11227,Granit Xhaka getting emotional after his subst...,"[{'text': 'afc', 'indices': [113, 117]}]",[❤️],Granit Xhaka,popular,negative,-0.6682,-0.7187
1067,goal,Everywhere,710,14768,Granit Xhaka scores the first goal in Arsenal'...,[],[❤️],Granit Xhaka,popular,negative,-0.6682,-0.7187
1066,piersmorgan,,748,10380,"Thank you and goodbye, Granit. We had a few wo...","[{'text': 'xhaka', 'indices': [225, 231]}, {'t...",[],Granit Xhaka,popular,negative,-0.6682,-0.7187
1065,ESPNFC,,765,12854,Granit Xhaka records a brace in 15 minutes in ...,[],"[🥺, 👏]",Granit Xhaka,popular,negative,-0.6682,-0.7187
